
# Tutorial Iterativo em Jupyter sobre Máquinas de Turing para Operações Aritméticas

<img src="./figuras/banner-lfac.png" alt="Banner" width="40%"/>


__Alan Lima Marques__

e-mail: [alanm@alunos.utfpr.edu.br](mailto:alanm@alunos.utfpr.edu.br)

Universidade Tecnológica Federal do Paraná (UTFPR)

Departamento Acadêmico de Computação (DACOM-CM)

Curso de Bacharelado em Ciência da Computação.

## Resumo

Este tutorial visa mostrar a implementação de máquinas de turing para operações aritméticas em python, utilizando a biblioteca [automata](https://pypi.org/project/automata-lib/).



## Introdução

Nesse relatório serão mostradas máquinas de Turing que realizam as quatro operações aritméticas básicas, são essas:
- Soma
- Subtração
- Multiplicação
- Divisão

As propriedades e a capacidade de uma máquina de Turing de poder navegar para a esquerda ou direita em sua fita (memória) tornam possível a realização dessas operações. O valor de entrada, que se encontra inicialmente na fita antes do processamento começar será dado em unário. Por exemplo, a entrada __#111+11#__ representa a operação __3+2__.


# Máquinas de Turing

A máquina de Turing é um modelo computacional teórico concebido por Alan turing, composto por uma fita de extensão infinita e um cabeçote de leitura/escrita. A máquina lê e escreve valores da fita, e movimenta-se para a esquerda, direita ou mantém-se na posição atual. Como outros tipos de autômatos, a máquina também tem estados e transições. 

![Representação](figuras/turing_machine.jpg)

__Figura 1:__ Representação de uma máquina de turing

Formalmente, uma máquina de Turing com uma fita pode ser definida da seguinte forma :
- Q   Conjunto de estados
- $\Sigma $   Alfabeto de símbolos 
- $\Gamma$   Alfabeto de fita
- $s\in Q$   Estado inicial
- $b\in \Gamma$   Símbolo de espaço em branco na fita
- $F \subseteq Q $   Conjunto de estados finais
- $\delta :Q\times \Gamma ⇒ Q\times \Gamma \times \{\leftarrow ,\rightarrow \}$   Função de transição 

Máquinas de Turing podem ter mais que uma fita, ou até fitas bi-dimensionais, mas o poder computacional dessas variações é o mesmo da máquina com uma só fita infinita para um dos lados. 

## Preparação do Ambiente

Para criação das Máquinas de Turing que realizam operações aritméticas precisaremos da biblioteca
[automata-lib](https://pypi.org/project/automata-lib/) que auxilia na implementação de Máquinas de Estados e Autômatos.

__O comando a seguir realizará a instalação da biblioteca:__


In [1]:
pip install automata-lib

Note: you may need to restart the kernel to use updated packages.


# Operações Aritméticas

A seguir será mostrado a implementação das operações aritméticas (soma, subtração, multiplicação e divisão ) no programa [JFlap](https://www.jflap.org/), seguido da sua implementação em python.

O artigo [Construction of a Basic Calculator through the Turing Machine – A Review¹](http://www.ijetajournal.org/volume-2/issue-6/IJETA-V2I6P1.pdf) mostra que é possível realizar essas quatro operações com máquinas de Turing de uma fita. 

## Adição

A máquina de Turing abaixo realiza a soma de dois números em unário. A entrada é feita no formato __#E+E__ e o resultado é a soma do número de E's. O autômato funciona substituindo o símbolo + por E e depois apagando o último E.


![Soma](figuras/soma.png)

__Figura 2:__ Representação de uma máquina de Turing que realiza soma.


A implementação da MT pode ser vista no código Soma

__Código Soma:__ Implementação da Máquina de Turing para Adição

In [2]:
from automata.tm.dtm import DTM
# Autômato que realiza a soma de dois números unários 
dtm_soma = DTM(
    states={'q0', 'q1', 'q2', 'q3', 'q4'},
    input_symbols={'E', '+','#'},
    tape_symbols={'E', '+', '#', ' '},
    transitions={
        'q0': {
            '#': ('q1', ' ', 'R')
        },
        'q1': {
            'E': ('q1', 'E', 'R'),
            '+': ('q2', 'E', 'R')
        },
        'q2': {
            'E': ('q2', 'E', 'R'),
            ' ': ('q3', ' ', 'L')
        },
        'q3': {
            'E': ('q4', ' ', 'N')
        }
    },
    initial_state='q0',
    blank_symbol=' ',
    final_states={'q4'}
)


Função de Validação para verificar se a especificação está correta.

In [3]:
dtm_soma.validate()  # returns True

True

O método/função `read_input(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna a configuração final para ela.

In [4]:
dtm_soma.read_input('#EEEEE+EEE').print()

q4:  EEEEEEEE  
             ^


O método/função `read_input_stepwise(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna todas as configurações da execução.

In [5]:
list(dtm_soma.read_input_stepwise('#EE+E')) # Lista todas as transições feitas pelo autômato.

[TMConfiguration('q0', TMTape('#EE+E', 0)),
 TMConfiguration('q1', TMTape(' EE+E', 1)),
 TMConfiguration('q1', TMTape(' EE+E', 2)),
 TMConfiguration('q1', TMTape(' EE+E', 3)),
 TMConfiguration('q2', TMTape(' EEEE', 4)),
 TMConfiguration('q2', TMTape(' EEEE ', 5)),
 TMConfiguration('q3', TMTape(' EEEE ', 4)),
 TMConfiguration('q4', TMTape(' EEE  ', 4))]

Abaixo alguns casos de teste:

In [6]:
inputs = ['#EE+E','#E+E','#EEEEE+EEEEEEEE','#E+','#+E']

for i in inputs:
    print("Verificando palavra:", i)
    if dtm_soma.accepts_input(i):
        print('accepted')
    else:
        print('rejected')
    dtm_soma.read_input(i).print()

Verificando palavra: #EE+E
accepted
q4:  EEE  
        ^
Verificando palavra: #E+E
accepted
q4:  EE  
       ^
Verificando palavra: #EEEEE+EEEEEEEE
accepted
q4:  EEEEEEEEEEEEE  
                  ^
Verificando palavra: #E+
accepted
q4:  E  
      ^
Verificando palavra: #+E
accepted
q4:  E  
      ^


## Subtração

A máquina de Turing abaixo realiza a subtração de dois números em unário. A entrada é feita no formato __#E-E#__ e o resultado é a subtração do número de __E's__.  O Autômato funciona removendo um símbolo E de cada lado do símbolo de subtração até que acabe os símbolos __E__ de um dos lados.

![Subtração](figuras/subtracao.png)

__Figura 3:__ Representação de uma máquina de turing que realiza subtração


A implementação da MT pode ser vista no código Subtração

__Código Subtração:__ Implementação da Máquina de Turing para Subtração

In [7]:
from automata.tm.dtm import DTM
# Autômato que realiza a subtração de dois números unários 
dtm_sub = DTM(
    states={'q0', 'q1', 'q2', 'q3', 'q4','q5','q6','q7','q8','q9'},
    input_symbols={'E', '-','#'},
    tape_symbols={'E', '-', '#', ' '},
    transitions={
        'q0': {
            '#': ('q1', '#', 'R')
        },
        'q1': {
            'E': ('q2', '#', 'R')            
        },
        'q2': {
            'E': ('q2', 'E', 'R'),
            '-': ('q3', '-', 'R')
        },
        'q3': {
            'E': ('q3', 'E', 'R'),
            '#': ('q4', '#', 'L')
        },
          'q4': {
            'E': ('q5', '#', 'L')
        },
          'q5': {
            '-': ('q9', '#', 'N'),
            'E': ('q6', 'E', 'L')
        },
          'q6': {
            'E': ('q6', 'E', 'L'),
            '-': ('q7', '-', 'L')
        },
          'q7': {
            'E': ('q8', 'E', 'L'),
            '#': ('q9', '#', 'N')
              
        },
          'q8': {
            'E': ('q8', 'E', 'L'),
            '#': ('q1', '#', 'R')
        }
        
    },
    initial_state='q0',
    blank_symbol='#', #se não der certo  ' '
    final_states={'q9'}
)


Função de Validação para verificar se a especificação está correta.

In [8]:
dtm_sub.validate()  # returns True

True

O método/função `read_input(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna a configuração final para ela.

In [9]:
dtm_sub.read_input('#EEEEE-EEE#').print()

q9: ####EE#####
          ^


O método/função `read_input_stepwise(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna todas as configurações da execução.

In [10]:
list(dtm_sub.read_input_stepwise('#EE-E#')) # Lista todas as transições feitas pelo autômato.

[TMConfiguration('q0', TMTape('#EE-E#', 0)),
 TMConfiguration('q1', TMTape('#EE-E#', 1)),
 TMConfiguration('q2', TMTape('##E-E#', 2)),
 TMConfiguration('q2', TMTape('##E-E#', 3)),
 TMConfiguration('q3', TMTape('##E-E#', 4)),
 TMConfiguration('q3', TMTape('##E-E#', 5)),
 TMConfiguration('q4', TMTape('##E-E#', 4)),
 TMConfiguration('q5', TMTape('##E-##', 3)),
 TMConfiguration('q9', TMTape('##E###', 3))]

Abaixo alguns casos de teste:

In [11]:
inputs = ['#EE-E#','#E-E#','#E-EE#','#EEEE-EEEEEE#']

for i in inputs:
    print("Verificando palavra:", i)
    if dtm_sub.accepts_input(i):
        print('accepted')
    else:
        print('rejected')
    dtm_sub.read_input(i).print()

Verificando palavra: #EE-E#
accepted
q9: ##E###
       ^
Verificando palavra: #E-E#
accepted
q9: #####
      ^
Verificando palavra: #E-EE#
accepted
q9: ##-E##
     ^
Verificando palavra: #EEEE-EEEEEE#
accepted
q9: #####-EE#####
        ^


O autômato não funciona nos casos onde o lado esquerdo ou direito da subtração são nulos. Para tal, transições adicionais nos estados q1 e q4 deveriam ser criadas, levando à um estado final.

## Multiplicação

A máquina de Turing abaixo realiza a multiplicação de dois números em unário. A entrada é feita no formato __1*1__ e o resultado é a multiplicação do número de símbolos __1__.  O Autômato funciona removendo um símbolo __1__ do lado esquerdo e repetindo a quantidade de __1__ do lado direito em forma de Z, que posteriormente são substituídos por símbolos __1__. Isso Significa que em uma multiplicação de __X*Y__ o autômato faz sucessivas __X__ somas de __Y__.   

![Multiplicação](figuras/multiplicacao.png)

__Figura 4:__ Representação de uma máquina de turing que realiza subtração.


A implementação da MT pode ser vista no código Multiplicação.

__Código Multiplicação:__ Implementação da Máquina de Turing para Multiplicação.

In [12]:
from automata.tm.dtm import DTM
# Autômato que realiza a multiplicação de dois números unários 
dtm_mult = DTM(
    states={'q0', 'q1', 'q2', 'q3', 'q4','q5','q6','q7','q8','q9','q10'},
    input_symbols={'1','*'},
    tape_symbols={'E', 'Z', '1', ' ','*'},
    transitions={
        'q0': {
            '1': ('q1', ' ', 'R')
        },
        'q1': {
            '*': ('q2', '1', 'R'),
            '1': ('q5', '1', 'R')
        },
        'q2': {
            '1': ('q2', '1', 'R'),
            'Z': ('q2', '1', 'R'),
            ' ': ('q3', ' ', 'L')
        },
        'q3': {
            '1': ('q4', ' ', 'N')
        },
          'q5': {
            '1': ('q5', '1', 'R'),
            '*': ('q6', '*', 'R')
        },
          'q6': {
            '1': ('q7', 'E', 'R'),
            'Z': ('q9', 'Z', 'L')
        },
          'q7': {
            'Z': ('q7', 'Z', 'R'),
            '1': ('q7', '1', 'R'),
            ' ': ('q8', 'Z', 'L')
        },
          'q8': {
            'Z': ('q8', 'Z', 'L'),
            '1': ('q8', '1', 'L'),
            'E': ('q6', 'E', 'R')
        },
          'q9': {
            'E': ('q9', '1', 'L'),
            '*': ('q10', '*', 'L')
        },
          'q10': {
            '1': ('q10', '1', 'L'),
            ' ': ('q0', ' ', 'R')
        }
        
    },
    initial_state='q0',
    blank_symbol=' ', #se não der certo  '#'
    final_states={'q4'}
)


Função de Validação para verificar se a especificação está correta.

In [13]:
dtm_mult.validate()  # returns True

True

O método/função `read_input(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna a configuração final para ela.

In [14]:
dtm_mult.read_input('111*11').print()

q4:    111111  
             ^


O método/função `read_input_stepwise(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna todas as configurações da execução.

In [15]:
list(dtm_mult.read_input_stepwise('11*11')) # Lista todas as transições feitas pelo autômato.

[TMConfiguration('q0', TMTape('11*11', 0)),
 TMConfiguration('q1', TMTape(' 1*11', 1)),
 TMConfiguration('q5', TMTape(' 1*11', 2)),
 TMConfiguration('q6', TMTape(' 1*11', 3)),
 TMConfiguration('q7', TMTape(' 1*E1', 4)),
 TMConfiguration('q7', TMTape(' 1*E1 ', 5)),
 TMConfiguration('q8', TMTape(' 1*E1Z', 4)),
 TMConfiguration('q8', TMTape(' 1*E1Z', 3)),
 TMConfiguration('q6', TMTape(' 1*E1Z', 4)),
 TMConfiguration('q7', TMTape(' 1*EEZ', 5)),
 TMConfiguration('q7', TMTape(' 1*EEZ ', 6)),
 TMConfiguration('q8', TMTape(' 1*EEZZ', 5)),
 TMConfiguration('q8', TMTape(' 1*EEZZ', 4)),
 TMConfiguration('q6', TMTape(' 1*EEZZ', 5)),
 TMConfiguration('q9', TMTape(' 1*EEZZ', 4)),
 TMConfiguration('q9', TMTape(' 1*E1ZZ', 3)),
 TMConfiguration('q9', TMTape(' 1*11ZZ', 2)),
 TMConfiguration('q10', TMTape(' 1*11ZZ', 1)),
 TMConfiguration('q10', TMTape(' 1*11ZZ', 0)),
 TMConfiguration('q0', TMTape(' 1*11ZZ', 1)),
 TMConfiguration('q1', TMTape('  *11ZZ', 2)),
 TMConfiguration('q2', TMTape('  111ZZ', 3)),
 

Abaixo alguns casos de teste:

In [16]:
inputs = ['1*1','1*111','1111*1','111*111','1*']

for i in inputs:
    print("Verificando palavra:", i)
    if dtm_mult.accepts_input(i):
        print('accepted')
    else:
        print('rejected')
    dtm_mult.read_input(i).print()

Verificando palavra: 1*1
accepted
q4:  1  
      ^
Verificando palavra: 1*111
accepted
q4:  111  
        ^
Verificando palavra: 1111*1
accepted
q4:     1111  
            ^
Verificando palavra: 111*111
accepted
q4:    111111111  
                ^
Verificando palavra: 1*
accepted
q4:    
     ^


O autômato funciona no caso __1*__ (1x0) mas não no caso __*1__ (0x1) pois não há uma transição em q0 para o símbolo __*__.

## Divisão

A máquina de Turing abaixo realiza a divisão de dois números em unário. A entrada é feita no formato __1/1__ e o resultado é a divisão inteira do número de símbolos  __1__ . O Autômato funciona removendo um símbolo __1__ do lado esquerdo e substituindo por __1__ por __E__ do lado direito a cada remoção bem sucedida. Quando não houver mais números __1__ do lado esquerdo, este adiciona um símbolo __Z__ à direta e troca todo __E__ por __1__ novamente, repetindo o processo enquanto houver símbolos à esquerda do símbolo de divisão. Em outras palavras, numa divisão __X/Y__ o autômato realiza sucessivas subtrações de __X__ por __Y__ , __Z__ vezes.  


![Divisão](figuras/divisao.png)

__Figura 5:__ Representação de uma máquina de turing que realiza divisão.


A implementação da MT pode ser vista no código Divisão.

__Código Divisão:__ Implementação da Máquina de Turing para Divisão.

In [17]:
from automata.tm.dtm import DTM
# Autômato que realiza a divisão de dois números unários 
dtm_div = DTM(
    states={'q1', 'q2', 'q3', 'q4','q5','q6','q7','q8','q9','q10'},
    input_symbols={'1','/'},
    tape_symbols={'E', 'Z', '1', ' ','/','#'},
    transitions={
        'q1': {
            '1': ('q4', ' ', 'R'),
            '/': ('q2', '#', 'R')
        },
        'q2': {
            'Z': ('q2', '1', 'R'),
            '1': ('q2', '#', 'R'),
            ' ': ('q3', ' ', 'N')
        },
        'q4': {
            '1': ('q4', '1', 'R'),
            '/': ('q5', '/', 'R')
        },
          'q5': {
            'E': ('q5', 'E', 'R'),
            'Z': ('q6', 'Z', 'L'),
            ' ': ('q6', ' ', 'L'),
            '1': ('q5', '1', 'R')
        },
          'q6': {
            'E': ('q6', 'E', 'L'),
            '1': ('q7', 'E', 'L')
        },
          'q7': {
            '/': ('q8', '/', 'R'),
            '1': ('q9', '1', 'L'),
        },
          'q8': {
            'Z': ('q8', 'Z', 'R'),
            'E': ('q8', '1', 'R'),
            ' ': ('q9', 'Z', 'L')
        },
          'q9': {
            'Z': ('q9', 'Z', 'L'),
            '/': ('q10', '/', 'L'),
            '1': ('q9', '1', 'L')
        },
          'q10': {
            '1': ('q10', '1', 'L'),
            ' ': ('q1', ' ', 'R')
        }
        
    },
    initial_state='q1',
    blank_symbol=' ', #se não der certo  '#'
    final_states={'q3'}
)


Função de Validação para verificar se a especificação está correta.

In [18]:
dtm_div.validate()  # returns True

True

O método/função `read_input(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna a configuração final para ela.

In [19]:
dtm_div.read_input('1111/11').print()

q3:     ###11 
             ^


O método/função `read_input_stepwise(palavra)` verifica se a _palavra_ é aceita pela Máquina de Turing e retorna todas as configurações da execução.

In [20]:
list(dtm_div.read_input_stepwise('11/11')) # Lista todas as transições feitas pelo autômato.

[TMConfiguration('q1', TMTape('11/11', 0)),
 TMConfiguration('q4', TMTape(' 1/11', 1)),
 TMConfiguration('q4', TMTape(' 1/11', 2)),
 TMConfiguration('q5', TMTape(' 1/11', 3)),
 TMConfiguration('q5', TMTape(' 1/11', 4)),
 TMConfiguration('q5', TMTape(' 1/11 ', 5)),
 TMConfiguration('q6', TMTape(' 1/11 ', 4)),
 TMConfiguration('q7', TMTape(' 1/1E ', 3)),
 TMConfiguration('q9', TMTape(' 1/1E ', 2)),
 TMConfiguration('q10', TMTape(' 1/1E ', 1)),
 TMConfiguration('q10', TMTape(' 1/1E ', 0)),
 TMConfiguration('q1', TMTape(' 1/1E ', 1)),
 TMConfiguration('q4', TMTape('  /1E ', 2)),
 TMConfiguration('q5', TMTape('  /1E ', 3)),
 TMConfiguration('q5', TMTape('  /1E ', 4)),
 TMConfiguration('q5', TMTape('  /1E ', 5)),
 TMConfiguration('q6', TMTape('  /1E ', 4)),
 TMConfiguration('q6', TMTape('  /1E ', 3)),
 TMConfiguration('q7', TMTape('  /EE ', 2)),
 TMConfiguration('q8', TMTape('  /EE ', 3)),
 TMConfiguration('q8', TMTape('  /1E ', 4)),
 TMConfiguration('q8', TMTape('  /11 ', 5)),
 TMConfigurat

Abaixo alguns casos de teste:

In [21]:
inputs = ['1/1','111/1','111111/111','/1']

for i in inputs:
    print("Verificando palavra:", i)
    if dtm_div.accepts_input(i):
        print('accepted')
    else:
        print('rejected')
    dtm_div.read_input(i).print()

Verificando palavra: 1/1
accepted
q3:  ##1 
        ^
Verificando palavra: 111/1
accepted
q3:    ##111 
            ^
Verificando palavra: 111111/111
accepted
q3:       ####11 
                ^
Verificando palavra: /1
accepted
q3: ## 
      ^


## Referências

Ezhilarasu P, __Construction of a Basic Calculator through the Turing Machine -A Review__. International Journal of Engineering Trends and Applications (IJETA), v. 2, 2015. Disponível em: http://www.ijetajournal.org/volume-2/issue-6/IJETA-V2I6P1.pdf Acesso em: 17 jun. 2022.
